In [1]:
import netCDF4 as nc
import numpy as np
from matplotlib import pyplot as plt
import os
import json
import pandas as pd
import seaborn as sns

In [2]:
!pwd

/work/scripts/caribou-poker-exp


In [3]:
os.environ['HDF5_USE_FILE_LOCKING']='FALSE'

In [4]:
run_name='BONA-black-spruce'

In [5]:
station_lat = 65.15401 #caribou creek
station_lon = -147.50258 #caribou creek

In [6]:
#get netcdf coordinates from runmask
runmask = nc.Dataset('/data/workflows/BONA-black-spruce/run-mask.nc')
lats=runmask.variables['lat'][:]
lons=runmask.variables['lon'][:]

In [7]:
#get distance between station and each cell
ydist = lats-station_lat
xdist = lons-station_lon
euc_dist = (ydist**2 + xdist**2)**.5

In [8]:
y_x = np.unravel_index(np.argmin(euc_dist),np.shape(lats))
print('y coordinate: {}'.format(y_x[0]))
print('x coordinate: {}'.format(y_x[1]))

y coordinate: 3
x coordinate: 0


In [9]:
%cd /work

/work


In [10]:
#set working directory
!scripts/setup_working_directory.py \
--input-data-path /data/input-catalog/caribou-poker/ \
/data/workflows/BONA-black-spruce/

Traceback (most recent call last):
  File "scripts/setup_working_directory.py", line 194, in <module>
    sys.exit(cmdline_entry())
  File "scripts/setup_working_directory.py", line 190, in cmdline_entry
    return cmdline_run(args)
  File "scripts/setup_working_directory.py", line 98, in cmdline_run
    shutil.copytree(os.path.join(ddt_dir, 'config'), os.path.join(args.new_directory, 'config'))
  File "/home/develop/.pyenv/versions/3.8.6/lib/python3.8/shutil.py", line 554, in copytree
    return _copytree(entries=entries, src=src, dst=dst, symlinks=symlinks,
  File "/home/develop/.pyenv/versions/3.8.6/lib/python3.8/shutil.py", line 455, in _copytree
    os.makedirs(dst, exist_ok=dirs_exist_ok)
  File "/home/develop/.pyenv/versions/3.8.6/lib/python3.8/os.py", line 223, in makedirs
    mkdir(name, mode)
FileExistsError: [Errno 17] File exists: '/data/workflows/BONA-black-spruce/config'


In [11]:
# Adjust the config file
CONFIG_FILE = os.path.join('/data/workflows/BONA-black-spruce/', 'config/config.js')
# Read the existing data into memory
with open(CONFIG_FILE, 'r') as f:
    config = json.load(f)
    
    config['IO']['output_nc_eq'] = 0 # Modify value...

# Write it back..
with open(CONFIG_FILE, 'w') as f:
    json.dump(config, f, indent=2)

In [12]:
!ls ../data/input-catalog/caribou-poker/

co2.nc		     historic-explicit-fire.nc	 run-mask.nc
drainage.nc	     projected-climate.nc	 soil-texture.nc
fri-fire.nc	     projected-co2.nc		 topo.nc
historic-climate.nc  projected-explicit-fire.nc  vegetation.nc


In [13]:
!ls ../data/workflows/BONA-black-spruce/

calibration  config  output  parameters  run-mask.nc


In [14]:
#poker flats: 0, 1
#caribou creek: 3, 0
# setup runmask
!runmask-util.py --reset \
--yx 3 0 \
--show \
/data/workflows/BONA-black-spruce/run-mask.nc

========== BEFORE ==================================
** Keep in mind that in this display the origin is the upper 
** left of the grid! This is opposite of the way that ncdump 
** and ncview display data (origin is lower left)!!

'/data/workflows/BONA-black-spruce/run-mask.nc'
<class 'netCDF4._netCDF4.Variable'>
int64 run(Y, X)
    grid_mapping: albers_conical_equal_area
unlimited dimensions: 
current shape = (4, 2)
filling on, default _FillValue of -9223372036854775806 used
[[0 0]
 [0 0]
 [0 0]
 [1 0]]

========== AFTER ==================================
** Keep in mind that in this display the origin is the upper 
** left of the grid! This is opposite of the way that ncdump 
** and ncview display data (origin is lower left)!!

'/data/workflows/BONA-black-spruce/run-mask.nc'
<class 'netCDF4._netCDF4.Variable'>
int64 run(Y, X)
    grid_mapping: albers_conical_equal_area
unlimited dimensions: 
current shape = (4, 2)
filling on, default _FillValue of -9223372036854775806 used
[[0 0]
 [0 

In [15]:
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --empty

In [16]:
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on CMTNUM yearly

In [17]:
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on GPP monthly

In [18]:
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on RG monthly  

In [19]:
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on RH monthly  

In [20]:
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on RM monthly

In [21]:
!scripts/outspec_utils.py ../data/workflows/BONA-black-spruce/config/output_spec.csv --on NPP monthly 

In [22]:
%cd /data/workflows/BONA-black-spruce

/data/workflows/BONA-black-spruce


In [23]:
!dvmdostem --force-cmt=13 --log-level='err' --tr-yrs=115 --sp-yrs=300 --eq-yrs=2000 --sc-yrs=85

Setting up logging...
[fatal] [] Skipping cell (0, 0)
[fatal] [] Skipping cell (0, 1)
[fatal] [] Skipping cell (1, 0)
[fatal] [] Skipping cell (1, 1)
[fatal] [] Skipping cell (2, 0)
[fatal] [] Skipping cell (2, 1)
[err] [PRE-RUN->Y] y: 3 x: 0 Year: 0
[err] [PRE-RUN->Y] y: 3 x: 0 Year: 1
[err] [PRE-RUN->Y] y: 3 x: 0 Year: 2
[err] [PRE-RUN->Y] y: 3 x: 0 Year: 3
[err] [PRE-RUN->Y] y: 3 x: 0 Year: 4
[err] [PRE-RUN->Y] y: 3 x: 0 Year: 5
[err] [PRE-RUN->Y] y: 3 x: 0 Year: 6
[err] [PRE-RUN->Y] y: 3 x: 0 Year: 7
[err] [PRE-RUN->Y] y: 3 x: 0 Year: 8
[err] [PRE-RUN->Y] y: 3 x: 0 Year: 9
[fatal] [EQ] Equilibrium Initial Year Count: 2000
[fatal] [EQ] Running Equilibrium, 2000 years.
[err] [EQ->Y] y: 3 x: 0 Year: 0
[err] [EQ->Y] y: 3 x: 0 Year: 1
[err] [EQ->Y] y: 3 x: 0 Year: 2
[err] [EQ->Y] y: 3 x: 0 Year: 3
[err] [EQ->Y] y: 3 x: 0 Year: 4
[err] [EQ->Y] y: 3 x: 0 Year: 5
[err] [EQ->Y] y: 3 x: 0 Year: 6
[err] [EQ->Y] y: 3 x: 0 Year: 7
[err] [EQ->Y] y: 3 x: 0 Year: 8
[err] [EQ->Y] y: 3 x: 0 Year: 9


[err] [EQ->Y] y: 3 x: 0 Year: 225
[err] [EQ->Y] y: 3 x: 0 Year: 226
[err] [EQ->Y] y: 3 x: 0 Year: 227
[err] [EQ->Y] y: 3 x: 0 Year: 228
[err] [EQ->Y] y: 3 x: 0 Year: 229
[err] [EQ->Y] y: 3 x: 0 Year: 230
[err] [EQ->Y] y: 3 x: 0 Year: 231
[err] [EQ->Y] y: 3 x: 0 Year: 232
[err] [EQ->Y] y: 3 x: 0 Year: 233
[err] [EQ->Y] y: 3 x: 0 Year: 234
[err] [EQ->Y] y: 3 x: 0 Year: 235
[err] [EQ->Y] y: 3 x: 0 Year: 236
[err] [EQ->Y] y: 3 x: 0 Year: 237
[err] [EQ->Y] y: 3 x: 0 Year: 238
[err] [EQ->Y] y: 3 x: 0 Year: 239
[err] [EQ->Y] y: 3 x: 0 Year: 240
[err] [EQ->Y] y: 3 x: 0 Year: 241
[err] [EQ->Y] y: 3 x: 0 Year: 242
[err] [EQ->Y] y: 3 x: 0 Year: 243
[err] [EQ->Y] y: 3 x: 0 Year: 244
[err] [EQ->Y] y: 3 x: 0 Year: 245
[err] [EQ->Y] y: 3 x: 0 Year: 246
[err] [EQ->Y] y: 3 x: 0 Year: 247
[err] [EQ->Y] y: 3 x: 0 Year: 248
[err] [EQ->Y] y: 3 x: 0 Year: 249
[err] [EQ->Y] y: 3 x: 0 Year: 250
[err] [EQ->Y] y: 3 x: 0 Year: 251
[err] [EQ->Y] y: 3 x: 0 Year: 252
[err] [EQ->Y] y: 3 x: 0 Year: 253
[err] [EQ->Y] 

[err] [EQ->Y] y: 3 x: 0 Year: 466
[err] [EQ->Y] y: 3 x: 0 Year: 467
[err] [EQ->Y] y: 3 x: 0 Year: 468
[err] [EQ->Y] y: 3 x: 0 Year: 469
[err] [EQ->Y] y: 3 x: 0 Year: 470
[err] [EQ->Y] y: 3 x: 0 Year: 471
[err] [EQ->Y] y: 3 x: 0 Year: 472
[err] [EQ->Y] y: 3 x: 0 Year: 473
[err] [EQ->Y] y: 3 x: 0 Year: 474
[err] [EQ->Y] y: 3 x: 0 Year: 475
[err] [EQ->Y] y: 3 x: 0 Year: 476
[err] [EQ->Y] y: 3 x: 0 Year: 477
[err] [EQ->Y] y: 3 x: 0 Year: 478
[err] [EQ->Y] y: 3 x: 0 Year: 479
[err] [EQ->Y] y: 3 x: 0 Year: 480
[err] [EQ->Y] y: 3 x: 0 Year: 481
[err] [EQ->Y] y: 3 x: 0 Year: 482
[err] [EQ->Y] y: 3 x: 0 Year: 483
[err] [EQ->Y] y: 3 x: 0 Year: 484
[err] [EQ->Y] y: 3 x: 0 Year: 485
[err] [EQ->Y] y: 3 x: 0 Year: 486
[err] [EQ->Y] y: 3 x: 0 Year: 487
[err] [EQ->Y] y: 3 x: 0 Year: 488
[err] [EQ->Y] y: 3 x: 0 Year: 489
[err] [EQ->Y] y: 3 x: 0 Year: 490
[err] [EQ->Y] y: 3 x: 0 Year: 491
[err] [EQ->Y] y: 3 x: 0 Year: 492
[err] [EQ->Y] y: 3 x: 0 Year: 493
[err] [EQ->Y] y: 3 x: 0 Year: 494
[err] [EQ->Y] 

[err] [EQ->Y] y: 3 x: 0 Year: 707
[err] [EQ->Y] y: 3 x: 0 Year: 708
[err] [EQ->Y] y: 3 x: 0 Year: 709
[err] [EQ->Y] y: 3 x: 0 Year: 710
[err] [EQ->Y] y: 3 x: 0 Year: 711
[err] [EQ->Y] y: 3 x: 0 Year: 712
[err] [EQ->Y] y: 3 x: 0 Year: 713
[err] [EQ->Y] y: 3 x: 0 Year: 714
[err] [EQ->Y] y: 3 x: 0 Year: 715
[err] [EQ->Y] y: 3 x: 0 Year: 716
[err] [EQ->Y] y: 3 x: 0 Year: 717
[err] [EQ->Y] y: 3 x: 0 Year: 718
[err] [EQ->Y] y: 3 x: 0 Year: 719
[err] [EQ->Y] y: 3 x: 0 Year: 720
[err] [EQ->Y] y: 3 x: 0 Year: 721
[err] [EQ->Y] y: 3 x: 0 Year: 722
[err] [EQ->Y] y: 3 x: 0 Year: 723
[err] [EQ->Y] y: 3 x: 0 Year: 724
[err] [EQ->Y] y: 3 x: 0 Year: 725
[err] [EQ->Y] y: 3 x: 0 Year: 726
[err] [EQ->Y] y: 3 x: 0 Year: 727
[err] [EQ->Y] y: 3 x: 0 Year: 728
[err] [EQ->Y] y: 3 x: 0 Year: 729
[err] [EQ->Y] y: 3 x: 0 Year: 730
[err] [EQ->Y] y: 3 x: 0 Year: 731
[err] [EQ->Y] y: 3 x: 0 Year: 732
[err] [EQ->Y] y: 3 x: 0 Year: 733
[err] [EQ->Y] y: 3 x: 0 Year: 734
[err] [EQ->Y] y: 3 x: 0 Year: 735
[err] [EQ->Y] 

[err] [EQ->Y] y: 3 x: 0 Year: 948
[err] [EQ->Y] y: 3 x: 0 Year: 949
[err] [EQ->Y] y: 3 x: 0 Year: 950
[err] [EQ->Y] y: 3 x: 0 Year: 951
[err] [EQ->Y] y: 3 x: 0 Year: 952
[err] [EQ->Y] y: 3 x: 0 Year: 953
[err] [EQ->Y] y: 3 x: 0 Year: 954
[err] [EQ->Y] y: 3 x: 0 Year: 955
[err] [EQ->Y] y: 3 x: 0 Year: 956
[err] [EQ->Y] y: 3 x: 0 Year: 957
[err] [EQ->Y] y: 3 x: 0 Year: 958
[err] [EQ->Y] y: 3 x: 0 Year: 959
[err] [EQ->Y] y: 3 x: 0 Year: 960
[err] [EQ->Y] y: 3 x: 0 Year: 961
[err] [EQ->Y] y: 3 x: 0 Year: 962
[err] [EQ->Y] y: 3 x: 0 Year: 963
[err] [EQ->Y] y: 3 x: 0 Year: 964
[err] [EQ->Y] y: 3 x: 0 Year: 965
[err] [EQ->Y] y: 3 x: 0 Year: 966
[err] [EQ->Y] y: 3 x: 0 Year: 967
[err] [EQ->Y] y: 3 x: 0 Year: 968
[err] [EQ->Y] y: 3 x: 0 Year: 969
[err] [EQ->Y] y: 3 x: 0 Year: 970
[err] [EQ->Y] y: 3 x: 0 Year: 971
[err] [EQ->Y] y: 3 x: 0 Year: 972
[err] [EQ->Y] y: 3 x: 0 Year: 973
[err] [EQ->Y] y: 3 x: 0 Year: 974
[err] [EQ->Y] y: 3 x: 0 Year: 975
[err] [EQ->Y] y: 3 x: 0 Year: 976
[err] [EQ->Y] 

...

[err] [EQ->Y] y: 3 x: 0 Year: 1419
[err] [EQ->Y] y: 3 x: 0 Year: 1420
[err] [EQ->Y] y: 3 x: 0 Year: 1421
[err] [EQ->Y] y: 3 x: 0 Year: 1422
[err] [EQ->Y] y: 3 x: 0 Year: 1423
[err] [EQ->Y] y: 3 x: 0 Year: 1424
[err] [EQ->Y] y: 3 x: 0 Year: 1425
[err] [EQ->Y] y: 3 x: 0 Year: 1426
[err] [EQ->Y] y: 3 x: 0 Year: 1427
[err] [EQ->Y] y: 3 x: 0 Year: 1428
[err] [EQ->Y] y: 3 x: 0 Year: 1429
[err] [EQ->Y] y: 3 x: 0 Year: 1430
[err] [EQ->Y] y: 3 x: 0 Year: 1431
[err] [EQ->Y] y: 3 x: 0 Year: 1432
[err] [EQ->Y] y: 3 x: 0 Year: 1433
[err] [EQ->Y] y: 3 x: 0 Year: 1434
[err] [EQ->Y] y: 3 x: 0 Year: 1435
[err] [EQ->Y] y: 3 x: 0 Year: 1436
[err] [EQ->Y] y: 3 x: 0 Year: 1437
[err] [EQ->Y] y: 3 x: 0 Year: 1438
[err] [EQ->Y] y: 3 x: 0 Year: 1439
[err] [EQ->Y] y: 3 x: 0 Year: 1440
[err] [EQ->Y] y: 3 x: 0 Year: 1441
[err] [EQ->Y] y: 3 x: 0 Year: 1442
[err] [EQ->Y] y: 3 x: 0 Year: 1443
[err] [EQ->Y] y: 3 x: 0 Year: 1444
[err] [EQ->Y] y: 3 x: 0 Year: 1445
[err] [EQ->Y] y: 3 x: 0 Year: 1446
[err] [EQ->Y] y: 3 x

[err] [EQ->Y] y: 3 x: 0 Year: 1654
[err] [EQ->Y] y: 3 x: 0 Year: 1655
[err] [EQ->Y] y: 3 x: 0 Year: 1656
[err] [EQ->Y] y: 3 x: 0 Year: 1657
[err] [EQ->Y] y: 3 x: 0 Year: 1658
[err] [EQ->Y] y: 3 x: 0 Year: 1659
[err] [EQ->Y] y: 3 x: 0 Year: 1660
[err] [EQ->Y] y: 3 x: 0 Year: 1661
[err] [EQ->Y] y: 3 x: 0 Year: 1662
[err] [EQ->Y] y: 3 x: 0 Year: 1663
[err] [EQ->Y] y: 3 x: 0 Year: 1664
[err] [EQ->Y] y: 3 x: 0 Year: 1665
[err] [EQ->Y] y: 3 x: 0 Year: 1666
[err] [EQ->Y] y: 3 x: 0 Year: 1667
[err] [EQ->Y] y: 3 x: 0 Year: 1668
[err] [EQ->Y] y: 3 x: 0 Year: 1669
[err] [EQ->Y] y: 3 x: 0 Year: 1670
[err] [EQ->Y] y: 3 x: 0 Year: 1671
[err] [EQ->Y] y: 3 x: 0 Year: 1672
[err] [EQ->Y] y: 3 x: 0 Year: 1673
[err] [EQ->Y] y: 3 x: 0 Year: 1674
[err] [EQ->Y] y: 3 x: 0 Year: 1675
[err] [EQ->Y] y: 3 x: 0 Year: 1676
[err] [EQ->Y] y: 3 x: 0 Year: 1677
[err] [EQ->Y] y: 3 x: 0 Year: 1678
[err] [EQ->Y] y: 3 x: 0 Year: 1679
[err] [EQ->Y] y: 3 x: 0 Year: 1680
[err] [EQ->Y] y: 3 x: 0 Year: 1681
[err] [EQ->Y] y: 3 x

[err] [EQ->Y] y: 3 x: 0 Year: 1889
[err] [EQ->Y] y: 3 x: 0 Year: 1890
[err] [EQ->Y] y: 3 x: 0 Year: 1891
[err] [EQ->Y] y: 3 x: 0 Year: 1892
[err] [EQ->Y] y: 3 x: 0 Year: 1893
[err] [EQ->Y] y: 3 x: 0 Year: 1894
[err] [EQ->Y] y: 3 x: 0 Year: 1895
[err] [EQ->Y] y: 3 x: 0 Year: 1896
[err] [EQ->Y] y: 3 x: 0 Year: 1897
[err] [EQ->Y] y: 3 x: 0 Year: 1898
[err] [EQ->Y] y: 3 x: 0 Year: 1899
[err] [EQ->Y] y: 3 x: 0 Year: 1900
[err] [EQ->Y] y: 3 x: 0 Year: 1901
[err] [EQ->Y] y: 3 x: 0 Year: 1902
[err] [EQ->Y] y: 3 x: 0 Year: 1903
[err] [EQ->Y] y: 3 x: 0 Year: 1904
[err] [EQ->Y] y: 3 x: 0 Year: 1905
[err] [EQ->Y] y: 3 x: 0 Year: 1906
[err] [EQ->Y] y: 3 x: 0 Year: 1907
[err] [EQ->Y] y: 3 x: 0 Year: 1908
[err] [EQ->Y] y: 3 x: 0 Year: 1909
[err] [EQ->Y] y: 3 x: 0 Year: 1910
[err] [EQ->Y] y: 3 x: 0 Year: 1911
[err] [EQ->Y] y: 3 x: 0 Year: 1912
[err] [EQ->Y] y: 3 x: 0 Year: 1913
[err] [EQ->Y] y: 3 x: 0 Year: 1914
[err] [EQ->Y] y: 3 x: 0 Year: 1915
[err] [EQ->Y] y: 3 x: 0 Year: 1916
[err] [EQ->Y] y: 3 x

[err] [SP->Y] y: 3 x: 0 Year: 129
[err] [SP->Y] y: 3 x: 0 Year: 130
[err] [SP->Y] y: 3 x: 0 Year: 131
[err] [SP->Y] y: 3 x: 0 Year: 132
[err] [SP->Y] y: 3 x: 0 Year: 133
[err] [SP->Y] y: 3 x: 0 Year: 134
[err] [SP->Y] y: 3 x: 0 Year: 135
[err] [SP->Y] y: 3 x: 0 Year: 136
[err] [SP->Y] y: 3 x: 0 Year: 137
[err] [SP->Y] y: 3 x: 0 Year: 138
[err] [SP->Y] y: 3 x: 0 Year: 139
[err] [SP->Y] y: 3 x: 0 Year: 140
[err] [SP->Y] y: 3 x: 0 Year: 141
[err] [SP->Y] y: 3 x: 0 Year: 142
[err] [SP->Y] y: 3 x: 0 Year: 143
[err] [SP->Y] y: 3 x: 0 Year: 144
[err] [SP->Y] y: 3 x: 0 Year: 145
[err] [SP->Y] y: 3 x: 0 Year: 146
[err] [SP->Y] y: 3 x: 0 Year: 147
[err] [SP->Y] y: 3 x: 0 Year: 148
[err] [SP->Y] y: 3 x: 0 Year: 149
[err] [SP->Y] y: 3 x: 0 Year: 150
[err] [SP->Y] y: 3 x: 0 Year: 151
[err] [SP->Y] y: 3 x: 0 Year: 152
[err] [SP->Y] y: 3 x: 0 Year: 153
[err] [SP->Y] y: 3 x: 0 Year: 154
[err] [SP->Y] y: 3 x: 0 Year: 155
[err] [SP->Y] y: 3 x: 0 Year: 156
[err] [SP->Y] y: 3 x: 0 Year: 157
[err] [SP->Y] 

[err] [TR->Y] y: 3 x: 0 Year: 70
[err] [TR->Y] y: 3 x: 0 Year: 71
[err] [TR->Y] y: 3 x: 0 Year: 72
[err] [TR->Y] y: 3 x: 0 Year: 73
[err] [TR->Y] y: 3 x: 0 Year: 74
[err] [TR->Y] y: 3 x: 0 Year: 75
[err] [TR->Y] y: 3 x: 0 Year: 76
[err] [TR->Y] y: 3 x: 0 Year: 77
[err] [TR->Y] y: 3 x: 0 Year: 78
[err] [TR->Y] y: 3 x: 0 Year: 79
[err] [TR->Y] y: 3 x: 0 Year: 80
[err] [TR->Y] y: 3 x: 0 Year: 81
[err] [TR->Y] y: 3 x: 0 Year: 82
[err] [TR->Y] y: 3 x: 0 Year: 83
[err] [TR->Y] y: 3 x: 0 Year: 84
[err] [TR->Y] y: 3 x: 0 Year: 85
[err] [TR->Y] y: 3 x: 0 Year: 86
[err] [TR->Y] y: 3 x: 0 Year: 87
[err] [TR->Y] y: 3 x: 0 Year: 88
[err] [TR->Y] y: 3 x: 0 Year: 89
[err] [TR->Y] y: 3 x: 0 Year: 90
[err] [TR->Y] y: 3 x: 0 Year: 91
[err] [TR->Y] y: 3 x: 0 Year: 92
[err] [TR->Y] y: 3 x: 0 Year: 93
[err] [TR->Y] y: 3 x: 0 Year: 94
[err] [TR->Y] y: 3 x: 0 Year: 95
[err] [TR->Y] y: 3 x: 0 Year: 96
[err] [TR->Y] y: 3 x: 0 Year: 97
[err] [TR->Y] y: 3 x: 0 Year: 98
[err] [TR->Y] y: 3 x: 0 Year: 99
[err] [TR-

In [24]:
!ls /data/workflows/BONA-black-spruce/output/

CMTNUM_yearly_sc.nc  NPP_monthly_sc.nc	RH_monthly_sc.nc  restart-eq.nc
CMTNUM_yearly_sp.nc  NPP_monthly_sp.nc	RH_monthly_sp.nc  restart-pr.nc
CMTNUM_yearly_tr.nc  NPP_monthly_tr.nc	RH_monthly_tr.nc  restart-sc.nc
GPP_monthly_sc.nc    RG_monthly_sc.nc	RM_monthly_sc.nc  restart-sp.nc
GPP_monthly_sp.nc    RG_monthly_sp.nc	RM_monthly_sp.nc  restart-tr.nc
GPP_monthly_tr.nc    RG_monthly_tr.nc	RM_monthly_tr.nc  run_status.nc


In [ ]:
deepc_ds = nc.Dataset('/data/workflows/BONA-black-spruce/output/DEEPC_yearly_eq.nc')
deepc = deepc_ds.variables['DEEPC'][:, y_x[0], y_x[1]]
year=np.array([i for i in range(0, len(deepc))]).astype(np.uint16)
deepc_df = pd.DataFrame({'year': year, 'DEEPC': deepc})

In [ ]:
sns.scatterplot(data=deepc_df, x='year', y='DEEPC')